##### Copyright 2025 Google LLC.

In [2]:
import csv
import easyocr
import shutil
from pathlib import Path
import pandas as pd
from PIL import Image
import PyPDF2
import sqlite3
import time
import sys
import atexit
import os
import json
from datetime import datetime
import locale
import platform
import configparser
import hashlib

# Attempt to import pyperclip for clipboard operations
try:
    import pyperclip
    CLIPBOARD_AVAILABLE = True
except ImportError:
    print("Warning: pyperclip library not found. Clipboard features will be unavailable.")
    print("Install it with: pip install pyperclip")
    CLIPBOARD_AVAILABLE = False

# --- Locale Setup ---
try:
    locale.setlocale(locale.LC_TIME, '')
except locale.Error:
    try:
        if platform.system() == 'Windows':
            locale.setlocale(locale.LC_TIME, 'English_United States.1252')
        else:
            locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')
    except locale.Error:
        locale.setlocale(locale.LC_TIME, 'C')





ModuleNotFoundError: No module named 'easyocr'

In [ ]:
def process_images_with_ocr(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    extracted_text = ""
    for detection in result:
        extracted_text += detection[1] + "\n"
    return extracted_text

In [ ]:
def convert_pdf_to_text(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [ ]:
# --- Configuration Loading ---
config = configparser.ConfigParser()
CONFIG_FILE = 'C:\\Users\\sabhi\\OneDrive\\Documents\\python projects\\other files\\config.ini'

def load_configuration():
    if os.path.exists(CONFIG_FILE):
        config.read(CONFIG_FILE)
        print(f"Configuration loaded from {CONFIG_FILE}")
    else:
        print(f"No {CONFIG_FILE} found. Creating default configuration.")
        config['GENERAL'] = {
            'log_file': 'activity_log.txt',
            'json_summary_log': 'session_summary.json',
            'max_backups': '20',
            'time_delay': '1.5'
        }
        with open(CONFIG_FILE, 'w') as f:
            config.write(f)
        print(f"Default {CONFIG_FILE} created. You can edit it to customize settings.")

    global log_file, json_summary_log, MAX_BACKUPS, time_delay
    log_file = config['GENERAL'].get('log_file', 'activity_log.txt')
    json_summary_log = config['GENERAL'].get('json_summary_log', 'session_summary.json')
    MAX_BACKUPS = int(config['GENERAL'].get('max_backups', '20'))
    time_delay = float(config['GENERAL'].get('time_delay', '1.5'))

load_configuration()

activity_summary = []

In [ ]:
# --- Global Variables ---
usage_counter = {
    "total_files": 0,
    "csv": 0, "tsv": 0,
    "database": 0,
    "text": 0, "log": 0,
    "binary": 0,
    "json": 0,
    "html": 0,
    "markdown": 0, "md": 0,
    "xml": 0,
    "yaml": 0, "yml": 0,
    "ini": 0, "conf": 0, "cfg": 0,
    "bat": 0, "sh": 0,
    "py": 0, "js": 0, "css": 0, "sql": 0,
    "dockerfile": 0,
    "cpp": 0, "java": 0, "go": 0, "rs": 0,
    "tsx": 0, "jsx": 0, "pl": 0
}

start_time = time.time()

def delay():
    time.sleep(time_delay)

In [ ]:
def log_action(action, skip_json_update=False):
    now = datetime.now()
    human_time = now.strftime("%A, %d %B %Y, %I:%M %p")
    with open(log_file, 'a') as lf:
        lf.write(f"({human_time}) {action}\n")
    activity_summary.append({"local_time": human_time, "action": action})
    if not skip_json_update:
        update_json_summary()

In [ ]:
def on_exit():
    end_time = time.time()
    mins, secs = divmod(int(end_time - start_time), 60)
    print(f"\nTotal Time Taken: {mins:02d}:{secs:02d}")
    log_action(f"Session ended. Total time: {mins:02d}:{secs:02d}")
    log_action(f"Total files created in session: {usage_counter['total_files']}, Breakdown: {usage_counter}")

atexit.register(on_exit)

def ask_overwrite(filepath):
    if os.path.exists(filepath):
        while True:
            choice = input(f"File '{filepath}' already exists. Overwrite? (yes/no): ").strip().lower()
            if choice in ['yes', 'no']:
                return choice == "yes"
            print("❗ Please enter 'yes' or 'no'.")
    return True

def backup_existing(filepath):
    if os.path.exists(filepath):
        for i in reversed(range(1, MAX_BACKUPS)):
            old = f"{filepath}.bak{i}"
            new = f"{filepath}.bak{i+1}"
            if os.path.exists(old):
                os.rename(old, new)
        os.rename(filepath, f"{filepath}.bak1")
        log_action(f"Backup created: {filepath}.bak1")

In [ ]:
def increment_usage(filetype):
    usage_counter["total_files"] += 1
    if filetype == "md": filetype = "markdown"
    elif filetype == "yml": filetype = "yaml"
    elif filetype in ["cfg", "conf"]: filetype = "ini"
    elif filetype == "sqlite": filetype = "database"
    if filetype in usage_counter:
        usage_counter[filetype] += 1
    else:
        if filetype in ['txt', 'log', 'css', 'js', 'sql', 'py', 'sh', 'bat', 'cpp', 'java', 'go', 'rs', 'tsx', 'jsx', 'pl']:
            usage_counter["text"] += 1
        else:
            usage_counter["binary"] += 1
        print(f"Warning: File type '{filetype}' not explicitly counted in usage. Counted as general.")
        log_action(f"Warning: File type '{filetype}' not explicitly counted. Counted as general.")

def validate_json(json_string):
    try:
        json.loads(json_string)
        return True
    except json.JSONDecodeError:
        return False

def validate_xml(xml_string):
    try:
        import xml.etree.ElementTree as ET
        ET.fromstring(xml_string)
        return True
    except Exception:
        return False

In [ ]:
def ask_directory_and_filename(default_name="untitled.txt"):
    while True:
        directory = input("Enter the directory path to save the file (or leave blank for current directory): ").strip()
        if not directory:
            directory = os.getcwd()
        if os.path.isdir(directory):
            break
        else:
            create = input(f"Directory '{directory}' does not exist. Create it? (yes/no): ").strip().lower()
            if create == 'yes':
                try:
                    os.makedirs(directory)
                    break
                except Exception as e:
                    print(f"❗ Failed to create directory: {e}")
            else:
                continue
    filename = input(f"Enter the filename (default: {default_name}): ").strip()
    if not filename:
        filename = default_name
    full_path = os.path.normpath(os.path.join(directory, filename))
    return full_path

In [ ]:
def file_processing():
    path = input("Enter the full path of the file or directory to process: ").strip().strip('"')
    path = os.path.normpath(path)
    if not os.path.exists(path):
        print(f"❗ Path does not exist: {path}")
        return
    if os.path.isfile(path):
        files_to_process = [path]
    elif os.path.isdir(path):
        files_to_process = [
            os.path.join(path, f)
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f))
        ]
    else:
        print("❗ Invalid path.")
        return
    output_format = input("Enter output file format (txt, csv, pdf, xlsx): ").strip().lower()
    output_filename = input("Enter the name of the output file (with extension): ").strip()
    output_dir = input("Enter the directory where the output file should be saved: ").strip().strip('"')
    if not output_dir:
        output_dir = os.getcwd()
    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
        except Exception as e:
            print(f"❗ Failed to create output directory: {e}")
            return
    output_path = os.path.normpath(os.path.join(output_dir, output_filename))
    for file_path in files_to_process:
        filename = os.path.basename(file_path)
        print(f"\nProcessing: {filename}")
        extracted_text = ""
        try:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                extracted_text = process_images_with_ocr(file_path)
            elif filename.lower().endswith('.pdf'):
                extracted_text = convert_pdf_to_text(file_path)
            elif filename.lower().endswith('.csv'):
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        csv_reader = csv.reader(file)
                        extracted_text = "\n".join([", ".join(row) for row in csv_reader])
                except UnicodeDecodeError:
                    with open(file_path, 'r', encoding='cp1252', errors='replace') as file:
                        csv_reader = csv.reader(file)
                        extracted_text = "\n".join([", ".join(row) for row in csv_reader])
            elif filename.lower().endswith('.xlsx'):
                import openpyxl
                workbook = openpyxl.load_workbook(file_path)
                sheet = workbook.active
                extracted_text = "\n".join(["\t".join([str(cell.value) for cell in row]) for row in sheet.iter_rows()])
            else:
                print(f"Skipping unsupported file type: {filename}")
                continue
            save_to_file(output_path, extracted_text, output_format)
            print(f"Saved processed data to {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    print("\n✅ Processing complete!")






In [ ]:
# --- File Creation ---
def create_file():
    print("\n--- Create New File ---")
    filetype = input("Enter file type (txt, md, json, xml, yaml, ini, html, py, js, css, sql, etc.): ").strip().lower()
    template = ""
    use_template = input(f"Use template for {filetype}? (yes/no): ").strip().lower()
    if use_template == "yes":
        template = suggest_template(filetype)
    content, used_clipboard = get_content_from_user_or_clipboard("Enter file content", filetype)
    if template and not content:
        content = template
    full_path = ask_directory_and_filename(f"untitled.{filetype}")
    if not ask_overwrite(full_path):
        print("File creation cancelled.")
        return
    backup_existing(full_path)
    try:
        with open(full_path, "w", encoding="utf-8") as f:
            f.write(content)
        increment_usage(filetype)
        log_action(f"Created file: {full_path} (type: {filetype})")
        print(f"✅ File created: {full_path}")
    except Exception as e:
        print(f"❗ Error creating file: {e}")


In [ ]:
# --- File Management ---
def manage_file():
    print("\n--- File Management ---")
    print("1. Delete file")
    print("2. Rename file")
    print("3. Copy file")
    print("4. Move file")
    print("5. Back to main menu")
    choice = input("Choose an option: ").strip()
    if choice == "1":
        path = input("Enter file path to delete: ").strip()
        if os.path.exists(path) and os.path.isfile(path):
            confirm = input(f"Are you sure you want to delete '{path}'? (yes/no): ").strip().lower()
            if confirm == "yes":
                try:
                    os.remove(path)
                    log_action(f"Deleted file: {path}")
                    print("✅ File deleted.")
                except Exception as e:
                    print(f"❗ Error deleting file: {e}")
        else:
            print("❗ File does not exist.")
    elif choice == "2":
        path = input("Enter file path to rename: ").strip()
        if os.path.exists(path) and os.path.isfile(path):
            new_name = input("Enter new filename: ").strip()
            new_path = os.path.join(os.path.dirname(path), new_name)
            try:
                os.rename(path, new_path)
                log_action(f"Renamed file: {path} -> {new_path}")
                print("✅ File renamed.")
            except Exception as e:
                print(f"❗ Error renaming file: {e}")
        else:
            print("❗ File does not exist.")
    elif choice == "3":
        src = input("Enter source file path: ").strip()
        dst = input("Enter destination file path: ").strip()
        try:
            shutil.copy2(src, dst)
            log_action(f"Copied file: {src} -> {dst}")
            print("✅ File copied.")
        except Exception as e:
            print(f"❗ Error copying file: {e}")
    elif choice == "4":
        src = input("Enter source file path: ").strip()
        dst = input("Enter destination file path: ").strip()
        try:
            shutil.move(src, dst)
            log_action(f"Moved file: {src} -> {dst}")
            print("✅ File moved.")
        except Exception as e:
            print(f"❗ Error moving file: {e}")
    else:
        return

In [ ]:
def manage_directory():
    print("\n--- Directory Management ---")
    print("1. Create directory")
    print("2. Delete directory")
    print("3. List directory contents")
    print("4. Back to main menu")
    choice = input("Choose an option: ").strip()
    if choice == "1":
        path = input("Enter directory path to create: ").strip()
        try:
            os.makedirs(path, exist_ok=True)
            log_action(f"Created directory: {path}")
            print("✅ Directory created.")
        except Exception as e:
            print(f"❗ Error creating directory: {e}")
    elif choice == "2":
        path = input("Enter directory path to delete: ").strip()
        if os.path.exists(path) and os.path.isdir(path):
            confirm = input(f"Are you sure you want to delete '{path}' and all its contents? (yes/no): ").strip().lower()
            if confirm == "yes":
                try:
                    shutil.rmtree(path)
                    log_action(f"Deleted directory: {path}")
                    print("✅ Directory deleted.")
                except Exception as e:
                    print(f"❗ Error deleting directory: {e}")
        else:
            print("❗ Directory does not exist.")
    elif choice == "3":
        path = input("Enter directory path to list: ").strip()
        if os.path.exists(path) and os.path.isdir(path):
            print("\nContents:")
            for item in os.listdir(path):
                print(" -", item)
        else:
            print("❗ Directory does not exist.")
    else:
        return

# --- Database Tools ---
def database_tools():
    print("\n--- Database Tools ---")
    print("1. Create SQLite database")
    print("2. Run SQL query")
    print("3. Back to main menu")
    choice = input("Choose an option: ").strip()
    if choice == "1":
        db_path = input("Enter path for new SQLite database: ").strip()
        try:
            conn = sqlite3.connect(db_path)
            conn.close()
            log_action(f"Created SQLite database: {db_path}")
            print("✅ Database created.")
        except Exception as e:
            print(f"❗ Error creating database: {e}")
    elif choice == "2":
        db_path = input("Enter SQLite database path: ").strip()
        if not os.path.exists(db_path):
            print("❗ Database file does not exist.")
            return
        query = input("Enter SQL query to execute: ").strip()
        try:
            conn = sqlite3.connect(db_path)
            cur = conn.cursor()
            cur.execute(query)
            if query.strip().lower().startswith("select"):
                rows = cur.fetchall()
                for row in rows:
                    print(row)
            else:
                conn.commit()
                print("✅ Query executed.")
            log_action(f"Executed SQL query on {db_path}: {query}")
            conn.close()
        except Exception as e:
            print(f"❗ Error executing query: {e}")
    else:
        return

In [3]:
# --- Advanced Tools ---
def advanced_tools():
    print("\n--- Advanced Tools ---")
    print("1. OCR Image to Text")
    print("2. PDF to Text")
    print("3. File Processing (batch OCR/PDF/CSV)")
    print("4. Back to main menu")
    choice = input("Choose an option: ").strip()
    if choice == "1":
        img_path = input("Enter image file path: ").strip()
        if os.path.exists(img_path):
            text = process_images_with_ocr(img_path)
            print("\nExtracted Text:\n", text)
            log_action(f"OCR processed image: {img_path}")
        else:
            print("❗ Image file does not exist.")
    elif choice == "2":
        pdf_path = input("Enter PDF file path: ").strip()
        if os.path.exists(pdf_path):
            text = convert_pdf_to_text(pdf_path)
            print("\nExtracted Text:\n", text)
            log_action(f"Converted PDF to text: {pdf_path}")
        else:
            print("❗ PDF file does not exist.")
    elif choice == "3":
        file_processing()
    else:
        return

def hash_file():
    print("\n--- File Hashing ---")
    file_path = input("Enter the file path to hash: ").strip()
    if not os.path.isfile(file_path):
        print("❗ File does not exist.")
        return
    algo = input("Choose hash algorithm (md5, sha1, sha256) [default: sha256]: ").strip().lower()
    if algo not in ["md5", "sha1", "sha256", ""]:
        print("❗ Unsupported algorithm.")
        return
    if not algo:
        algo = "sha256"
    try:
        h = hashlib.new(algo)
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                h.update(chunk)
        print(f"{algo.upper()} hash: {h.hexdigest()}")
        log_action(f"Hashed file {file_path} with {algo}")
    except Exception as e:
        print(f"❗ Error hashing file: {e}")

def search_in_files():
    print("\n--- Search in Files ---")
    directory = input("Enter directory to search in: ").strip()
    if not os.path.isdir(directory):
        print("❗ Directory does not exist.")
        return
    pattern = input("Enter text pattern to search for: ").strip()
    found = False
    for root, _, files in os.walk(directory):
        for fname in files:
            fpath = os.path.join(root, fname)
            try:
                with open(fpath, "r", encoding="utf-8", errors="ignore") as f:
                    for i, line in enumerate(f, 1):
                        if pattern in line:
                            print(f"Found in {fpath} (line {i}): {line.strip()}")
                            found = True
            except Exception:
                continue
    if not found:
        print("No matches found.")
    else:
        log_action(f"Searched for '{pattern}' in {directory}")


In [4]:
def view_log():
    print("\n--- View Activity Log ---")
    if os.path.exists(log_file):
        with open(log_file, "r", encoding="utf-8", errors="replace") as f:
            print(f.read())
    else:
        print("No log file found.")

def main_menu_extended():
    while True:
        print("\n==== Main Menu (Extended) ====")
        print("1. Create File")
        print("2. Manage File")
        print("3. Manage Directory")
        print("4. Database Tools")
        print("5. Advanced Tools")
        print("6. Hash File")
        print("7. Search in Files")
        print("8. View Activity Log")
        print("9. Exit")
        choice = input("Choose an option: ").strip()
        if choice == "1":
            delay()
            create_file()
        elif choice == "2":
            delay()
            manage_file()
        elif choice == "3":
            delay()
            manage_directory()
        elif choice == "4":
            delay()
            database_tools()
        elif choice == "5":
            delay()
            advanced_tools()
        elif choice == "6":
            delay()
            hash_file()
        elif choice == "7":
            delay()
            search_in_files()
        elif choice == "8":
            delay()
            view_log()
        elif choice == "9":
            delay()
            log_action("User exited the program.")
            print("Exiting program...")
            break
        else:
            print("❗ Invalid choice. Please try again.")

if __name__ == "__main__":
    load_configuration()
    print("Welcome to the File Management System!")
    print(f"Configuration loaded from {CONFIG_FILE}")
    print(f"Log file: {log_file}, JSON summary log: {json_summary_log}")
    print(f"Max backups: {MAX_BACKUPS}, Time delay: {time_delay} seconds")
    main_menu_extended()


NameError: name 'load_configuration' is not defined